In [1]:
import pandas
import datetime
import numpy
from davitpy.models import *
from davitpy import utils
%matplotlib inline

In [2]:
inpTecFile = "../data/gps120618g.001.hdf5"
dataDF = pandas.read_hdf(inpTecFile, 'Data/Table Layout')
dataDF["date"] = pandas.to_datetime(dataDF["year"]*10000000000 +\
                                    dataDF["month"]*100000000 + dataDF["day"]*1000000 +\
                                    dataDF["hour"]*10000 + dataDF["min"]*100 +\
                                    dataDF["sec"],format='%Y%m%d%H%M%S')
dataDF.head()

,year,month,day,hour,min,sec,ut1_unix,ut2_unix,recno,gdlat,glon,tec,dtec,date
0,2012,6,18,0,2,30,1339977600,1339977900,2,-89.0,-175.0,3.7,1.5,2012-06-18 00:02:30
1,2012,6,18,0,2,30,1339977600,1339977900,2,-89.0,-174.0,3.6,1.4,2012-06-18 00:02:30
2,2012,6,18,0,2,30,1339977600,1339977900,2,-89.0,-173.0,3.2,1.4,2012-06-18 00:02:30
3,2012,6,18,0,2,30,1339977600,1339977900,2,-89.0,-172.0,3.2,1.4,2012-06-18 00:02:30
4,2012,6,18,0,2,30,1339977600,1339977900,2,-89.0,-171.0,3.2,1.4,2012-06-18 00:02:30


In [ ]:
def convert_to_mlt(row):
    mlt, mlat = utils.coord_conv( row['glon'], row['gdlat'], \
                                 "geo", "mlt", altitude=300., \
                                 date_time=row['date'] )
    return mlt, mlat

dataDF["mlt"], dataDF["mlat"] = zip( *dataDF.apply( convert_to_mlt, axis=1 ) )

dataDF.head()